In [6]:
# coding: utf-8

import matplotlib.pyplot as plt
import datetime
import os

import numpy as np

import skimage.io
import skimage.transform

import tensorflow as tf
import tensorflow.contrib.slim as slim

import inception
import pickle
import math

from detnet import det_net, det_net_loss
import nnutil
import volleyball
from volleyball import *


In [7]:
## config
class Config(object):

  def __init__(self):

    # shared
    self.image_size = 720, 1280
    self.out_size = 87, 157
    self.batch_size = 4
    self.num_boxes = 12
    self.epsilon = 1e-5
    self.features_multiscale_names = ['Mixed_5d', 'Mixed_6e']
    self.train_inception = False

    # DetNet
    self.build_detnet = False
    self.num_resnet_blocks = 1
    self.num_resnet_features = 512
    self.reg_loss_weight = 10.0
    self.nms_kind = 'greedy'

    # ActNet
    self.use_attention = False
    self.crop_size = 5, 5
    self.num_features_boxes = 4096
    self.num_actions = 9
    self.num_activities = 8
    self.actions_loss_weight = 4.0
    self.actions_weights = [[ 1., 1., 2., 3., 1., 2., 2., 0.2, 1.]]

    # sequence
    self.num_features_hidden = 1024
    self.num_frames = 10
    self.num_before = 5
    self.num_after = 4

    # training parameters
    self.train_num_steps = 5000
    self.train_random_seed = 0
    self.train_learning_rate = 1e-5
    self.train_dropout_prob = 0.8
    self.train_save_every_steps = 500

src_model_config = 'models/activity/volleyball/config-temporal-actions-2048-5x5-weight-0.5-match-hidden-soft.pkl'

# we load config from the first stage
c = pickle.load(open(src_model_config, 'rb'))
c.src_model_path = c.out_model_path
c.ckpt_dir = 'models/activity/volleyball/'
c.data_path = '../Data/videos'
print(c.data_path)

../Data/videos


In [8]:
## reading the dataset
train = volley_read_dataset(c.data_path, TRAIN_SEQS + VAL_SEQS)
train_frames = volley_all_frames(train)
test = volley_read_dataset(c.data_path, TEST_SEQS)
test_frames = volley_all_frames(test)

all_anns = {**train, **test}
all_tracks = pickle.load(open(c.data_path + '/tracks_normalized.pkl', 'rb')) #a dict of dicts containing the bboxes
print(len(test_frames))
print(len(train_frames))

1337
3493


In [9]:
# data loading utils
def _frames_around(frame, num_before=5, num_after=4):
  sid, src_fid = frame
  return [(sid, src_fid, fid)
          for fid in range(src_fid-num_before, src_fid+num_after+1)]

def load_samples_sequence(anns, tracks, images_path, frames, num_boxes=12):
  # NOTE: this assumes we got the same # of boxes for this batch
  images, boxes, boxes_idx = [], [], []
  activities, actions = [], []
  for i, (sid, src_fid, fid) in enumerate(frames):
    images.append(skimage.io.imread(images_path + '/%d/%d/%d.jpg' %
                                    (sid, src_fid, fid)))

    boxes.append(tracks[(sid, src_fid)][fid])
    actions.append(anns[sid][src_fid]['actions'])
    if len(boxes[-1]) != num_boxes:
      boxes[-1] = np.vstack([boxes[-1], boxes[-1][:num_boxes-len(boxes[-1])]])
      actions[-1] = actions[-1] + actions[-1][:num_boxes-len(actions[-1])]
    boxes_idx.append(i * np.ones(num_boxes, dtype=np.int32))
    activities.append(anns[sid][src_fid]['group_activity'])


  images = np.stack(images)
  activities = np.array(activities, dtype=np.int32)
  bboxes = np.vstack(boxes).reshape([-1, num_boxes, 4])
  bboxes_idx = np.hstack(boxes_idx).reshape([-1, num_boxes])
  actions = np.hstack(actions).reshape([-1, num_boxes])

  return images, activities, bboxes, bboxes_idx, actions



In [15]:
## testing
tf.reset_default_graph()

tf_config = tf.ConfigProto()
tf_config.log_device_placement = True
tf_config.gpu_options.allow_growth = True
tf_config.allow_soft_placement = True

np.random.seed(c.train_random_seed) #can use same random seed as for training
tf.set_random_seed(c.train_random_seed)
num_iter = int(math.ceil(len(test_frames) / c.batch_size))

with tf.Session(config=tf_config) as sess:
  saver = tf.train.import_meta_graph(c.out_model_path + '.meta')
  saver.restore(sess, c.out_model_path)
  """
  ckpt = tf.train.get_checkpoint_state(c.ckpt_dir)
  if ckpt and ckpt.model_checkpoint_path:
    # Restores from checkpoint
    saver.restore(sess, ckpt.model_checkpoint_path)
    # Assuming model_checkpoint_path looks something like:
    #   /my-favorite-path/cifar10_train/model.ckpt-0,
    # extract global_step from it.
    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
  else:
    print('No checkpoint file found')
    """
    
  #print('loading pre-trained model...')
  
  #restorer = tf.train.Saver(inception_vars + shared_vars)
  #restorer.restore(sess, c.src_model_path)
  #print('done!')

  #print('initializing the variables...')
  #sess.run(tf.initialize_variables(sequence_vars +
  #                                 train_vars))
  #print('done!')

  #saver = tf.train.Saver()

  fetches = [
    actions_accuracy,
    actions_preds,
    activities_accuracy,
    activities_preds,
  ]

  evaluations_store = []
  for step in range(1, num_iter):
    p = range(c.batch_size*(step-1),c.batch_size*step)
    batch_frames = sum([_frames_around(test_frames[i],
                                       c.num_before,
                                       c.num_after)
                        for i in p], [])
    batch = load_samples_sequence(all_anns, all_tracks, c.images_path, batch_frames)
    batch = [b.reshape((c.batch_size,c.num_frames) + b.shape[1:]) for b in batch]

    print(batch[0].shape)
    feed_dict = {
      images_in : batch[0],
      activities_in : batch[1],
      boxes_in : batch[2],
      boxes_idx_in : batch[3],
      actions_in : batch[4],
      dropout_keep_prob_in : c.train_dropout_prob,
    }

    outputs = sess.run(fetches, feed_dict)
    evaluations_store.append(outputs)
    if step % 5 == 0:
      ts = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
      print('%s step:%d ' % (ts, outputs[-1]) +
            'actv.A: %.4f ' % (outputs[2]) +
            'actn.A: %.4f ' % (outputs[0]))
  print(mean(evaluations_store))

    
  print('done!')


INFO:tensorflow:Restoring parameters from models/activity/volleyball/stage-b-temporal-actions-2048-5x5-weight-0.5-match-hidden-soft.ckpt


NameError: name 'actions_accuracy' is not defined

In [18]:
## testing
tf.reset_default_graph()

tf_config = tf.ConfigProto()
tf_config.log_device_placement = True
tf_config.gpu_options.allow_growth = True
tf_config.allow_soft_placement = True

np.random.seed(c.train_random_seed) #can use same random seed as for training
tf.set_random_seed(c.train_random_seed)
num_iter = int(math.ceil(len(test_frames) / c.batch_size))

with tf.Session(config=tf_config) as sess:
  saver = tf.train.import_meta_graph(c.out_model_path + '.meta')
  saver.restore(sess, c.out_model_path)
  print(sess.run('InceptionV3/Mixed_7c/Branch_2/Conv2d_0d_3x1/weights:0'))
  print(sess.run('actions_accuracy:0'))
    

INFO:tensorflow:Restoring parameters from models/activity/volleyball/stage-b-temporal-actions-2048-5x5-weight-0.5-match-hidden-soft.ckpt
[[[[-1.48251904e-02 -2.44097263e-02 -1.04602342e-02 ...  5.28921839e-03
    -2.38534738e-03  8.12498110e-05]
   [-2.96938466e-03  5.43707190e-03  1.32095469e-02 ...  8.52427678e-04
     1.39422268e-02 -1.54105630e-02]
   [-1.99191552e-02 -6.53531263e-03  1.36847468e-02 ... -2.09240941e-03
     8.76897015e-03 -4.14402271e-03]
   ...
   [ 1.03049260e-02 -9.63514205e-03  1.97985629e-03 ... -2.71806139e-02
    -2.72925827e-03 -2.63223262e-03]
   [-1.64683554e-02 -1.36193270e-02  1.04020536e-02 ... -1.76987480e-02
    -9.56962258e-03  2.05083415e-02]
   [ 6.94127055e-03 -2.06818189e-02  1.19229977e-03 ...  7.88421778e-04
    -1.15041304e-02  7.00882496e-03]]]


 [[[-1.73342917e-02 -1.67243648e-02 -9.03176703e-03 ...  2.99323909e-03
    -1.08495355e-04  1.78871502e-03]
   [-3.24645196e-03  9.68667772e-03  1.02207605e-02 ... -1.76786468e-03
     1.22127375e-

ValueError: Fetch argument 'actions_accuracy:0' cannot be interpreted as a Tensor. ("The name 'actions_accuracy:0' refers to a Tensor which does not exist. The operation, 'actions_accuracy', does not exist in the graph.")

In [12]:
import os
print(c.ckpt_dir)
print(os.listdir(c.ckpt_dir))
print(tf.train.latest_checkpoint(c.ckpt_dir))
print(c.out_model_path)

models/activity/volleyball/
['config-single-actions-2048-5x5-weight-0.5.pkl', 'config-temporal-actions-2048-5x5-weight-0.5-match-hidden-soft.pkl', 'stage-b-temporal-actions-2048-5x5-weight-0.5-match-hidden-soft.ckpt.data-00000-of-00001', 'stage-b-temporal-actions-2048-5x5-weight-0.5-match-hidden-soft.ckpt.index', 'stage-b-temporal-actions-2048-5x5-weight-0.5-match-hidden-soft.ckpt.meta']
None
models/activity/volleyball/stage-b-temporal-actions-2048-5x5-weight-0.5-match-hidden-soft.ckpt


In [17]:
from tensorflow.python.tools import inspect_checkpoint as chkp

# print all tensors in checkpoint file
chkp.print_tensors_in_checkpoint_file(c.out_model_path,tensor_name='', all_tensors=True)

tensor_name:  ActNet/sequence/actions_hidden/fully_connected/biases
[ 0.00070825 -0.00187222 -0.00149594 -0.00078     0.00095532 -0.00041063
 -0.00101914  0.00245381  0.00072683]
tensor_name:  ActNet/sequence/actions_hidden/fully_connected/weights
[[ 0.04384461 -0.03579939  0.06801098 ... -0.00915553  0.05789946
  -0.03174346]
 [ 0.00435904 -0.03138454  0.00611335 ...  0.04918373 -0.05578678
   0.03126673]
 [ 0.04229015  0.01571766  0.01251065 ... -0.02572684 -0.04964579
   0.0293777 ]
 ...
 [-0.00968678  0.05800711  0.01838336 ...  0.02722861 -0.03591804
  -0.00259253]
 [-0.06890176 -0.002471    0.03375063 ...  0.00625785 -0.01582255
  -0.06258738]
 [-0.04385801 -0.05961612 -0.07304663 ...  0.02877679 -0.04140683
  -0.06930996]]
tensor_name:  ActNet/sequence/activities_hidden/fully_connected/biases
[ 4.3178041e-04  6.5639470e-05  1.0023180e-04 -5.0256326e-04
  1.8120804e-04  1.4732196e-05 -1.6834626e-04 -5.6794268e-04]
tensor_name:  ActNet/sequence/activities_hidden/fully_connected/we

    -0.00416449]]]]
tensor_name:  InceptionV3/Mixed_6a/Branch_1/Conv2d_0b_3x3/BatchNorm/beta
[-1.1967719  -1.1404897  -0.6886775  -0.47550818 -1.053246   -0.44995826
 -0.5924706  -1.0742888  -0.7341693  -0.47215006 -0.96437955 -1.2009997
 -0.6368767  -0.80001056 -0.51394635 -0.87791383 -0.60408264 -0.65761787
 -0.5849324  -0.56925005 -0.78482866 -0.8524349  -1.0498575  -0.78895044
 -0.9750202  -0.90527755 -0.9318101  -1.1293758  -0.8979207  -0.69721985
 -0.9917511  -1.0091774  -0.90635705 -0.8619536  -0.77730393 -0.91591305
 -0.68874335 -0.95663863 -1.0439302  -0.6868532  -0.950241   -0.5245235
 -0.82447827 -0.76981634 -0.44926867 -0.8964908  -0.81134456 -1.1186152
 -1.0915301  -0.9373303  -0.7906448  -0.9970777  -0.8664697  -1.2791357
 -0.5974479  -0.6724435  -1.1516465  -1.1540573  -0.85452116 -1.5077333
 -1.3593065  -1.249463   -0.54844284 -1.0659581  -1.0358318  -0.87193656
 -0.9383436  -1.9713635  -0.74888325 -0.88601375 -0.93700695 -0.82310903
 -0.8023798  -0.7188082  -0.8861939 

 -0.866788   -0.7360308  -0.27942502 -0.7019015 ]
tensor_name:  InceptionV3/Mixed_6d/Branch_2/Conv2d_0a_1x1/BatchNorm/moving_variance
[0.19535695 0.21732888 0.28917927 0.23228744 0.29771495 0.3378248
 0.17546193 0.19719246 0.28539023 0.19521227 0.37545422 0.41935617
 0.24499944 0.3673952  0.2999645  0.28091222 0.2868898  0.29594588
 0.5431548  0.42719585 0.19118455 0.30533728 0.28114462 0.2692682
 0.25565958 0.2504298  0.27318865 0.18962249 0.1641008  0.32161433
 0.2876832  0.7442883  0.2181146  0.30720773 0.30729622 0.234604
 0.3007651  0.8699646  0.29417327 0.38029382 0.19614086 0.3251627
 0.36819616 0.6174479  0.1897092  0.28134885 0.98618907 0.31954232
 0.16413738 0.3425834  0.32628977 0.2109615  0.20105419 0.16649367
 0.38861343 0.34535593 0.20100152 0.32984287 0.67177814 0.51727283
 0.22361785 0.42278105 0.21055636 0.24054621 0.6093466  0.45611775
 0.31721896 0.25163567 0.24995461 0.34381104 0.17956348 0.28136992
 0.2719005  0.3430703  0.49658623 0.37942225 0.23835622 0.56771636


 -0.5711842  -0.7342423  -0.69642866 -0.6041847  -0.7462352  -0.805175  ]
tensor_name:  InceptionV3/Mixed_7b/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_mean
[-2.03099623e-01 -1.70769915e-01 -1.86461717e-01 -2.81425208e-01
 -1.24789372e-01  7.46171921e-02 -1.28906995e-01  4.55859192e-02
 -2.14804798e-01  1.09045736e-01 -3.14702094e-02 -1.43233120e-01
 -2.26973966e-01 -4.63897213e-02 -1.84933946e-01 -3.18830967e-01
 -1.47087708e-01 -5.55030107e-02 -7.67124444e-02  1.00505307e-01
 -2.87123710e-01 -1.36262402e-01 -1.43446505e-01 -4.09656242e-02
 -6.08741008e-02 -2.37121508e-01 -3.00420463e-01 -2.57552117e-01
 -2.24942252e-01 -1.66151643e-01 -1.36015162e-01 -3.16519022e-01
  6.27166480e-02 -8.38999450e-02 -1.95772380e-01 -1.31609961e-01
 -2.38636807e-01 -1.78203043e-02 -5.49936376e-04  3.44955102e-02
 -1.55715555e-01 -1.06360041e-01  8.47806036e-02 -2.97973841e-01
 -2.80238926e-01  1.69657201e-01 -1.54954895e-01 -9.29383188e-02
 -3.67955029e-01 -1.20950833e-01 -1.17130212e-01 -3.86081301e-02
 

 0.00846551 0.00799574 0.00852776 0.00808512 0.00757551 0.00841947]
tensor_name:  InceptionV3/Mixed_7c/Branch_1/Conv2d_0c_3x1/weights
[[[[-1.99434091e-03 -5.68171497e-03 -9.20287333e-03 ...  3.64443241e-03
    -8.39834660e-03 -8.92269891e-03]
   [-1.04201129e-02 -7.10596982e-03  6.87622814e-04 ... -1.69464108e-02
    -5.20585384e-03  8.39666114e-04]
   [ 8.44813958e-02 -2.91242148e-03  3.08047072e-03 ...  2.95620020e-02
    -1.34000322e-04  7.31341774e-03]
   ...
   [-1.77240483e-02 -6.43552514e-03 -1.21766534e-02 ...  1.09474584e-02
     4.76769246e-02 -1.01828692e-03]
   [ 3.54017988e-02  6.53556827e-03 -2.74223974e-04 ...  2.32670200e-03
     9.31562111e-03 -2.25419970e-03]
   [-1.36765707e-02 -6.46140240e-03  1.07242051e-03 ...  7.55378231e-03
    -3.57380975e-03  1.83823693e-03]]]


 [[[-5.58336172e-03  1.59179431e-03 -6.85886526e-03 ... -9.30602546e-05
    -7.72844534e-03 -3.17235105e-03]
   [-1.06763327e-02 -8.10140930e-03 -2.41541630e-03 ... -1.27068665e-02
    -1.62216311e-03 

[[7.2196840e-09 2.1894750e-09 5.0974833e-09 ... 4.4018735e-09
  6.0985608e-09 7.5062268e-10]
 [2.4413935e-09 5.5168967e-09 5.8193725e-09 ... 6.3766783e-09
  6.9527641e-09 9.1249974e-10]
 [5.1122364e-09 6.2611369e-09 1.6345654e-08 ... 6.7287971e-09
  1.4781339e-08 1.5224122e-09]
 ...
 [7.9409288e-09 7.8685138e-09 6.4440182e-09 ... 1.1177424e-08
  1.5660182e-08 2.1499322e-09]
 [7.9490361e-09 1.2381481e-08 9.2025010e-09 ... 1.8293891e-08
  2.1655525e-08 3.8666177e-09]
 [8.3288390e-09 1.3091262e-08 4.8687716e-09 ... 5.8060787e-09
  1.5777680e-08 1.5877799e-09]]
tensor_name:  train/OptimizeLoss/ActNet/shared/fully_connected/biases/Adam
[-7.6291966e-05  3.9930466e-05  7.0668517e-05 ... -7.3016970e-05
 -1.0030157e-04  1.3349368e-04]
tensor_name:  train/OptimizeLoss/ActNet/shared/fully_connected/biases/Adam_1
[1.9046270e-07 1.0409108e-07 1.4875008e-07 ... 3.9398390e-07 3.3129328e-07
 2.2254761e-07]
tensor_name:  train/OptimizeLoss/ActNet/shared/fully_connected/weights/Adam
[[-2.1475040e-05  4.